# DIC Crack Inspector

In [ ]:
%matplotlib widget
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from io import BytesIO
import importlib
import bmcs_shear.dic_crack.dic_test_report as reporter

In [ ]:
import matplotlib.pylab as plt
import matplotlib.gridspec as gridspec
from bmcs_shear.dic_crack import sz_tests_series_2023 as ts
import numpy as np
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

In [ ]:
tests = [ts.B1_TV1, ts.B1_TV2, ts.B6_TV1, 
         # ts.B6_TV2, 
         ts.B7_TV1, ts.B7_TV2, 
         ts.B8_TV1, ts.B8_TV2, ts.B9_TV1, ts.B9_TV2, ts.B10_TV1, ts.B10_TV2]

In [ ]:
def fig_test_subplots():
    fig = plt.figure(figsize=(10, 3))
    fig.canvas.header_visible = False
    grid = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 1], height_ratios=[0.4])
    ax_list = fig.add_subplot(grid[0, :])
    return fig, ax_list

def fig_crack_t_subplots():
    fig = plt.figure(figsize=(10, 3.5))
    fig.canvas.header_visible = False
    grid = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 1], height_ratios=[1])
    ax_list = [fig.add_subplot(grid[0, i]) for i in range(3)]
    return fig, ax_list

def fig_cor_t_subplots():
    fig, ax = plt.subplots(1,1, figsize=(10, 3))
    fig.canvas.header_visible = False
    fig.tight_layout()
    return fig, ax


In [ ]:
active_tests = tests
dcl_tests = [ts.new_dcl(test) for test in active_tests]
dcl_tests

In [ ]:
def plot_crack_detection_field(dcl, ax, fig, t_detect=None):
    if t_detect is None:
        t_detect = dcl.t_detect
    dcl.dsf.plot_crack_detection_field(ax, fig)
    ax.set_aspect("equal")

Set up the reporter

In [ ]:
def grid_shared_axes(axs):
    m, n = axs.shape
    # Hide axis labels and ticks on all axes
    for ax in axs.flat:
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    # Make specific axes visible
    for i in range(0, m):
        axs[i, 0].get_yaxis().set_visible(True)
    for j in range(0, n):
        ax = axs[-1, j] 
        ax.get_xaxis().set_visible(True)
        ax.set_xlim(xmin=0)

In [ ]:
importlib.reload(reporter)

report = reporter.DICReportTemplate(
    output_dir='.')

n_tests = len(dcl_tests)
figs = []
specimen_names = []

ts_test_setup = (r'''
    \subsection{Design parameters}
    ''' )

ts_DIC_load_history = (r'''
    \subsection{DIC import and load history parameters}
    ''')

ts_interpolator = (r'''
    \subsection{Interpolator parameters}
    Parameters of the base grid constructed from unstructured data. This grid serves for the 
    evaluation of strain field and the fields derived based on constitutive assumptions, i.e.
    damage, stress, and energy release fields.  
    ''')

ts_crack_detection = (r'''
    \subsection{Crack detection parameters}
    ''' )

ts_crack_kinematics = r'''
    \subsection{Crack kinematics}

    The three diagrams show the values of maximum compressive strain at the top of the section, 
    the maximum crack opening and sliding at the bottom of the section. The shape of the curves can be 
    used as an indicator of the initiating failure mechanism.  
    '''
ts_tooth_kinematics = r'''
    \subsection{Tooth kinematics}
    The left diagram shows the evaluated centers of rotation are plotted for the crack detection load level 
    $t_\mathrm{detect} \in (0,1)$. This load level marks the end of the primary crack propagation that appears to comply 
    with the rotational tooth kinematics around a crack. Once the a dowel failure appears 
    in one of the cracks, a unique center of rotation cannot be found.  
    '''
ts_M_phi = r'''
    The following diagram uses the above displayed centers of rotation evaluated for the whole loading history 
    as a basis for the quantification of the relation between the bending 
    moment and rotation between two neighboring teeth. Note that the maximum value of the 
    moment decreases from the left to the right corresponding to the linear profile 
    of the bending moment along the length $a$.    
    '''

m_rows = 3
for i, dcl in enumerate(dcl_tests):
    print(f'test {i}')
    dic_grid = dcl.dsf.dic_grid

    report.add_section(dcl.dir_name)
    report.add_text(ts_test_setup
                    + dic_grid.dic_inp.get_latex_design_params())

    # Specimen - setup
    dic_grid.t = 1.1
    fig, ax = fig_test_subplots()
    # dcl.bd.plot_sz_bd(ax)
    plot_crack_detection_field(dcl, ax, fig)
    #dcl.dsf.plot_eps_IJab(ax)
    dic_grid.sz_bd.plot_sz_bd(ax)
    # dcl.plot_primary_cracks(ax_cracks=ax)
    ax.set_title(f'FE-DIC damage at $F_{{\max}}$',
        loc='left')
    ax.set_aspect("equal")
    ax.title.set_position([.2, 0.6])
    fig.subplots_adjust(wspace=0.3, hspace=0.4)
    report.add_figure(fig)
    plt.close(fig)

    report.add_text(
        r'''\subsection{Energy release rate}
        Hot spots of energy release due to the strain localization in active cracks 
        is evaluated using the term
        \begin{align}
        \dot{Y} = \frac{1}{2} \varepsilon_{ij} D_{ijkl} \varepsilon{kl} \dot{\omega}
        \end{align}
        ''')
    fig_energy_release, axs = plt.subplots(5,2,figsize=(10,9))
    for i, t in enumerate([0.5, 0.75, 0.9, 1, 1.1]):
        dic_grid.t = t
        dcl.dsf.plot_eps_IJab(axs[i, 0])
        axs[i,0].set_title(f'$F = ${dic_grid.F_T_t:.0f} kN', pad=0)
        dcl.dsf.plot_crack_detection_field(axs[i, 1])
        dcl.dsf.plot_dY_t_IJ(axs[i, 1], t)
        axs[i,1].set_title(f'$\\approx t =${dic_grid.t:.2f}', pad=0)
        axs[i,1].axis('equal')
        axs[i,1].axis('off')
    # fig_energy_release.tight_layout()
    fig_energy_release.subplots_adjust(wspace=0.01, hspace=0.15)
    report.add_figure(fig_energy_release, caption='''
        Localization process; Left: Evolution of absolute maximum principle values of the strain fields
                      divided into the tensile and compression regions; Right: Damage field in 
                      greyscale with values above 0.3 in white and energy release rate showing the active 
                      process zones in the form of red burning hot spots.
                      ''')
    plt.close(fig_energy_release)

    # Crack detection state
    report.add_text(ts_DIC_load_history
                    + '''The relative position of the DIC window and the applied
                    padding with respect to the boundary of the exported point cloud
                    is given in the following table.''' 
                    + dic_grid.dic_inp.get_latex_dic_params())
    report.add_text(ts_interpolator 
                    + dic_grid.get_latex_grid_params())
    report.add_text(ts_crack_detection 
                    + '''The parameter values applied in the crack detection algorithm 
                    of the current test are summarized in the following table.
                    '''
                    + dcl.get_latex_detection_params())

    fig_crack_detection = plt.figure(figsize=(10, 9))
    axs = gridspec.GridSpec(5, 2, width_ratios=[4, 1], height_ratios=[1, 1, 1, 1, 1])
    axs.update(wspace=0.02, hspace=0.1)
    fig_crack_detection.canvas.header_visible = False
    for i, t in enumerate([0.5, 0.75, 0.9, 1, 1.1]):
        dic_grid.t = t
        ax_cracks = fig_crack_detection.add_subplot(axs[i, 0])
        ax_ld = fig_crack_detection.add_subplot(axs[i, 1])
        dcl.dsf.plot_sig_compression_field(ax_cracks)
        dcl.dsf.plot_dY_t_IJ(ax_cracks, t)
        dcl.plot_primary_cracks(ax_cracks=ax_cracks)
        dcl.plot_crack_roots(ax_cracks)
        ax_cracks.set_aspect("equal")
        dic_grid.dic_inp.plot_load_deflection(ax_ld)
    fig_crack_detection.subplots_adjust(wspace=0.05, hspace=0.1)
    report.add_figure(fig_crack_detection, 
                      caption='''Cracking process; Left: Detected enumerated crack states plotted against the blue 
                      contours of the compression stress field; 
                      Right: Load deflection curve with a marked crack detection load level.   
                      ''')
    plt.close(fig_crack_detection)

    report.add_text(r'''
        \subsection{Breakdown of stress transfer mechanisms versus equilibrium} 
    ''')
    # Create a grid of axes with shared axis definitions
    n = len(dcl.cracks)
    m = m_rows
    cell_size = 10 // n
    m_size, n_size = m*cell_size, n*cell_size
    fig_ST, axs = plt.subplots(m, n, sharex='col', sharey='row', figsize=(n_size, m_size*1.2))
    fig_ST.canvas.header_visible=False
    n_C = min(len(dcl.cracks), 9)
    for C in range(n_C):
        print('C', C)
        cr = dcl.cracks[C]
        cr.sp.plot_ST(*axs[:,C], legend=False)
        # Adjust the spacing to eliminate gaps between the diagrams
    grid_shared_axes(axs)
    fig_ST.subplots_adjust(wspace=0, hspace=0)
    report.add_figure(
        fig_ST,
        caption='''
        Equilibrium and stress transfer mechanisms
    ''',
    )
    plt.close(fig_ST)

    report.add_text(ts_crack_kinematics)
    fig_cr, (ax_cr_eps, ax_cr_w, ax_cr_s) = fig_crack_t_subplots()
    F_T = dic_grid.dic_inp.F_T
    t_T = dic_grid.t_T
    for C, cr in enumerate(dcl.cracks):
        sp = cr.sp
        eps_Tab = np.min(cr.get_eps_Kab(t_T, cr.X_1_Ka[-30:,:]), axis=1)
        eps_T00 = eps_Tab[:, 0, 0]
        ax_cr_eps.plot(eps_T00, F_T, color=cr.color)
        # plot kinematics
        u_Ta = np.max(cr.get_u_crc_Ka(t_T, cr.X_crc_1_Ka[:5,:]), axis=1)
        ax_cr_w.plot(sp.u_TNa[:, 0, 0], F_T, label=f'crack {cr.name}', color=cr.color)        
        ax_cr_s.plot(sp.u_TNa[:, 0, 1], F_T, label=f'crack {C+1}', color=cr.color)
    # decoration
    ax_cr_eps.set_ylabel(r'$F$/kN')
    ax_cr_eps.set_xlabel(r'$\varepsilon_{\mathrm{top}}$/-')
    ax_cr_eps.set_title('Load vs. max. compression strain')
    ax_cr_eps.grid()
    # decoration
    ax_cr_w.set_ylabel(r'$F$/kN')
    ax_cr_w.set_xlabel(r'$w_{\mathrm{bottom}}$/mm')
    ax_cr_w.set_title('Load vs. max. crack opening')
    ax_cr_w.grid()
    # decoration
    ax_cr_s.set_ylabel(r'$F$/kN')
    ax_cr_s.set_xlabel(r'$s_{\mathrm{bottom}}$/mm')
    ax_cr_s.set_title('Load vs. max. slip')
    ax_cr_s.grid()
    ax_cr_s.legend()
    fig_cr.subplots_adjust(wspace=0.3, hspace=0.4)
    report.add_figure(fig_cr)
    plt.close(fig_cr)

    report.add_text(ts_tooth_kinematics)
    fig_cor, ax_cor_loc = fig_cor_t_subplots()
    dic_grid.t = dcl.t_detect
    dic_grid.plot_bounding_box(ax_cor_loc)
    for cr in dcl.cracks:
        cr.plot_X_t_Ka(ax_cor_loc)
        cr.cor._plot_crack_cor(ax_cor_loc)
    ax_cor_loc.set_axis_off()
    ax_cor_loc.set_aspect('equal')
    ax_cor_loc.set_title(f'Center of rotation at $F$ = {dcl.t_detect}$F_{{\max}}$')
    fig_cor.subplots_adjust(wspace=0.3, hspace=0.4)
    report.add_figure(fig_cor)
    plt.close(fig_cor)

    report.add_text(ts_M_phi)
    fig_M_phi, ax_M_phi = plt.subplots(1,1, figsize=(6,4))
    M_Ct, phi_Ct = np.einsum('CMt->MCt', dcl.M_phi_Ct)
    phi_Ct = np.rad2deg(phi_Ct)
    for cr, M_t, phi_t in zip(dcl.cracks, M_Ct, phi_Ct):
        ax_M_phi.plot(phi_t, M_t, label=f'crack {cr.name}', color=cr.color)
    ax_M_phi.set_ylabel(r'$M$/kNm')
    ax_M_phi.set_xlabel(r'$\phi$/deg')
    ax_M_phi.set_title('Bending moment vs. tooth rotation')
    ax_M_phi.grid()
    report.add_figure(fig_M_phi, width=0.5)
    plt.close(fig_M_phi)

report.generate_report()